## Лабораторная работа №6

### Задание: Провести классификацию найденного датасета, методами CatBoost.

In [5]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Загрузка датасета
dataset_train = pd.read_csv("train.csv")

# Предварительная обработка данных
dataset_train['Age'].fillna(dataset_train['Age'].median(), inplace=True)
dataset_train['Embarked'].fillna(dataset_train['Embarked'].mode()[0], inplace=True)
dataset_train['Fare'].fillna(dataset_train['Fare'].median(), inplace=True)

# Преобразование категориальных переменных в числовые
dataset_train = pd.get_dummies(dataset_train, columns=['Sex', 'Embarked'], drop_first=True)

# Выбор признаков и целевой переменной
X = dataset_train[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S']]
y = dataset_train['Survived']

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение и оценка модели CatBoost
catboost_model = CatBoostClassifier(random_seed=42, silent=True)
catboost_model.fit(X_train, y_train)
catboost_pred = catboost_model.predict(X_test)

catboost_accuracy = accuracy_score(y_test, catboost_pred)
print("Точность CatBoost:", catboost_accuracy)
print("\nConfusion Матрица CatBoost:\n", confusion_matrix(y_test, catboost_pred))
print("\nClassification Отчет CatBoost:\n", classification_report(y_test, catboost_pred))

# Оптимизация гиперпараметров с использованием кросс-валидации
param_grid = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'iterations': [100, 200, 300]
}

grid_search = GridSearchCV(estimator=CatBoostClassifier(random_seed=42, silent=True), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Лучшие параметры для CatBoost:", grid_search.best_params_)
print("Лучший результат кросс-валидации для CatBoost:", grid_search.best_score_)

# Обучение и оценка модели CatBoost с оптимальными параметрами
best_catboost_model = grid_search.best_estimator_
best_catboost_pred = best_catboost_model.predict(X_test)

best_catboost_accuracy = accuracy_score(y_test, best_catboost_pred)
print("Точность CatBoost с оптимальными параметрами:", best_catboost_accuracy)
print("\nConfusion Матрица CatBoost с оптимальными параметрами:\n", confusion_matrix(y_test, best_catboost_pred))
print("\nClassification Отчет CatBoost с оптимальными параметрами:\n", classification_report(y_test, best_catboost_pred))


Точность CatBoost: 0.8268156424581006

Confusion Матрица CatBoost:
 [[96  9]
 [22 52]]

Classification Отчет CatBoost:
               precision    recall  f1-score   support

           0       0.81      0.91      0.86       105
           1       0.85      0.70      0.77        74

    accuracy                           0.83       179
   macro avg       0.83      0.81      0.82       179
weighted avg       0.83      0.83      0.82       179

Лучшие параметры для CatBoost: {'depth': 6, 'iterations': 100, 'learning_rate': 0.2}
Лучший результат кросс-валидации для CatBoost: 0.8370530877573131
Точность CatBoost с оптимальными параметрами: 0.8100558659217877

Confusion Матрица CatBoost с оптимальными параметрами:
 [[94 11]
 [23 51]]

Classification Отчет CatBoost с оптимальными параметрами:
               precision    recall  f1-score   support

           0       0.80      0.90      0.85       105
           1       0.82      0.69      0.75        74

    accuracy                         